In [6]:
#import the required libraries for our project
import tensorflow as tf
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout,Activation
from keras.optimizers import SGD
import numpy as np
import json
import pickle
import pandas as pd
import random

lemma=WordNetLemmatizer()


#intialize the objects
words=[]
classes=[]
documents=[]
ignore_words=['?','!']
#Open the intent file and store it in a variable
data_intent = open("/content/intents.json").read()
intents=json.loads(data_intent)

In [9]:
#Preprocess the data in the intents file we say tokenization
nltk.download('punkt')
for intent in intents["intents"]:
  for pattern in intent["patterns"]:

    #tokenize the words and append to the variable word
    w= nltk.word_tokenize(pattern)
    words.extend(w)

    #add documents in the corpus
    documents.append((w, intent["tag"]))

    #add to our classes list
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
#Now we will lemmatize each word and remove duplicate words from the list
nltk.download('wordnet')
words= [lemma.lemmatize(w.lower()) for w in words if w not in ignore_words]

#sorted the words
words=sorted(list(set(words)))

#sort classes
classes=sorted(list(set(classes)))

#Print the len of the classes
print(len(classes),"classes",classes)

#print the documents length
print(len(documents),"documents")

#print the length of the words
print(len(words),"Unqiue lemmatize words",words)

#Dump the word file and classes file
pickle.dump(words,open("words_file.pickle","wb"))
pickle.dump(classes,open("classes_file.pickle","wb"))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
19 classes ['bye', 'greeting', 'help', 'instructions', 'mood_happy', 'mood_unhappy', 'organs', 'register', 'safety', 'side_effects', 'spread', 'symptoms', 'thanks', 'treatment', 'vaccine', 'vaccine_safe', 'what', 'where', 'worldwidecases']
84 documents
103 Unqiue lemmatize words ["'m", '19', 'about', 'affected', 'afternoon', 'allright', 'anyone', 'anything', 'are', 'available', 'avoid', 'awesome', 'bad', 'be', 'being', 'birth', 'by', 'bye', 'can', 'care', 'case', 'coivd', 'common', 'contracting', 'corona', 'coronavirus', 'could', 'covid', 'covid-19', 'covid19', 'cu', 'did', 'do', 'effect', 'evening', 'existing', 'explain', 'feel', 'fine', 'first', 'for', 'good', 'hello', 'help', 'helpful', 'here', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'information', 'instruction', 'is', 'kind', 'know', 'link', 'list', 'lost', 'lovely', 'me', 'morning', 'most', 'nice', 'not', 'of', 'ok', 'organ', 

In [16]:
#Creat training and testing data
training_data=[]

#Create a empty array for the output data
output_data= [0] * len(classes)

#training set and bag of words each set
for doc in documents:
  #intilaize the bag of words
  bag= []
  #list of tokenize words
  pattern_words= doc[0]

  pattern_words = [lemma.lemmatize(word.lower()) for word in pattern_words]

  #create our bag of words array with 1
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  output_row= list(output_data)
  output_row[classes.index(doc[1])] = 1

  training_data.append([bag,output_row])

random.shuffle(training_data)
training_data = np.array(training_data)
#Create a train and test lists
train_x= list(training_data[:,0])
train_y= list(training_data[:,1])

print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [18]:
training_data[0]

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
      dtype=object)

In [21]:
#Build the model

model=Sequential()
#Add the first layer with 128 neurons
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
#Add the dropout layer
model.add(Dropout(0.5))
#Add the second layer
model.add(Dense(64,activation="relu"))
#Add a dropout layer
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation="softmax"))


#Compile the model
sgd=SGD(learning_rate=0.01,momentum=0.9,nesterov=True)

model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])


In [23]:
#Fit and save the model
hits= model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)


Epoch 1/200
17/17 [==============================] - 0s 2ms/step - loss: 1.8569 - accuracy: 0.4643
Epoch 2/200
17/17 [==============================] - 0s 2ms/step - loss: 1.8006 - accuracy: 0.4762
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 1.6058 - accuracy: 0.5000
Epoch 4/200
17/17 [==============================] - 0s 2ms/step - loss: 1.6423 - accuracy: 0.5238
Epoch 5/200
17/17 [==============================] - 0s 2ms/step - loss: 1.4134 - accuracy: 0.5238
Epoch 6/200
17/17 [==============================] - 0s 2ms/step - loss: 1.5134 - accuracy: 0.5595
Epoch 7/200
17/17 [==============================] - 0s 2ms/step - loss: 1.0579 - accuracy: 0.7143
Epoch 8/200
17/17 [==============================] - 0s 2ms/step - loss: 1.1595 - accuracy: 0.6905
Epoch 9/200
17/17 [==============================] - 0s 2ms/step - loss: 0.9286 - accuracy: 0.7857
Epoch 10/200
17/17 [==============================] - 0s 2ms/step - loss: 0.8563 - accuracy: 0.7738
Epoch 11/

In [24]:
#Save the model
model.save("chatbot_model.h5",hits)